In [4]:
from lib import dataset_manager, exploratory_analysis, linear_regression

# LIXO

In [10]:
%load_ext autoreload
%autoreload 2

# Ficha técnica

## Alunos

### Francesco Brandão Ferreira de Carli

Matrícula: ...

Email: ...

### Guilherme de Oliveira Silva

Matrícula: 2018058490

Email: guilhermeoliveiras@ufmg.br

## Tema
Análise municipal dos impactos de infraestrutura de Internet na taxa de aprovação de alunos de escolas públicas

## Apresentação
(Link para o vídeo)

# Datasets

descricao dos datasets

In [5]:
dataset = dataset_manager.get_working_dataset()
dataset.head()

Loading and cleaning IBGE data...
Done
Loading and cleaning Internet access data...
Done
Loading education data...
Done
Loading social indicators...
Done


,cod_mun,nom_mun,sig_uf,regiao,tx_aprov_ef_1,saeb_mat_ef_1,saeb_port_ef_1,ideb_ef_1,tx_aprov_ef_2,saeb_mat_ef_2,...,type,num_operadoras,0_a_2_mbs,2.5_a_5_mbs,6_a_10_mbs,11_a_15_mbs,15_a_25_mbs,acima_25_mbs,0_a_10_mbs,acima_10_mbs
0,1100015,ALTA FLORESTA D'OESTE,RO,NORTE,92.4,226.44,204.70,5.7,95.3,270.02,...,broadband,3.0,0.001066,0.000000,0.000000,0.000000,0.000000,0.000902,0.001066,0.000902
1,1100023,ARIQUEMES,RO,NORTE,88.7,214.18,199.92,5.2,93.7,266.09,...,broadband,7.0,0.000598,0.001040,0.000952,0.000133,0.000697,0.001793,0.002590,0.002623
2,1100023,ARIQUEMES,RO,NORTE,88.7,214.18,199.92,5.2,93.7,266.09,...,mobile,1.0,0.000000,0.000155,0.000266,0.000354,0.000000,0.000000,0.000421,0.000354
3,1100049,CACOAL,RO,NORTE,98.0,223.45,209.48,5.8,98.3,267.51,...,broadband,5.0,0.000738,0.000916,0.000000,0.000000,0.000535,0.005396,0.001654,0.005931
4,1100049,CACOAL,RO,NORTE,98.0,223.45,209.48,5.8,98.3,267.51,...,mobile,3.0,0.000204,0.000407,0.000000,0.000407,0.000000,0.000000,0.000611,0.000407


# Análise exploratória

## Dados de acesso à Internet

### Visão geral

As variáveis a seguir foram coletadas a partir do mapa do [SIMET](https://simet.nic.br/mapas-app.html). A plataforma disponibiliza estatísticas de testes de velocidade de acesso à Internet para uma pequena região e categoriza os resultados em:

1. operadora do serviço
2. faixa de velocidade, em Mb/s
3. tipo de conexão: `mobile` (redes móveis) ou `broadband` (banda larga fixa)

De posse dos dados, agregamos as estatísticas coletadas por município e tipo de conexão, contabilizando o número de operadoras únicas e o total de dispositivos por faixa. Para as variáveis de dispositivos por faixa, ainda normalizamos os resultados pela população, a fim de tornar as regiões comparáveis entre si – isto é, a variável está na unidade *dispositivos na faixa n por habitante*. Assim, a variável `type` é categórica, enquanto que as variáveis `tcp_range_<n>` (`<n>` representa a faixa de velocidade) e `num_operadoras` são numéricas – respectivamente, contínuas e discreta.

In [6]:
exploratory_analysis.get_dataset_description(dataset, 'internet')

,0_a_2_mbs,2.5_a_5_mbs,6_a_10_mbs,11_a_15_mbs,15_a_25_mbs,acima_25_mbs,0_a_10_mbs,acima_10_mbs
count,4843.000000,4843.000000,4843.000000,4843.000000,4843.000000,4843.000000,4843.000000,4843.000000
mean,0.000903,0.000351,0.000555,0.000636,0.000800,0.001963,0.001829,0.003399
std,0.001383,0.000762,0.001185,0.001200,0.001680,0.004819,0.002605,0.006783
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000228,0.000000
50%,0.000412,0.000000,0.000000,0.000000,0.000000,0.000000,0.001053,0.001216
75%,0.001291,0.000397,0.000714,0.000891,0.000990,0.001991,0.002328,0.003602
max,0.018041,0.012561,0.027686,0.016920,0.019868,0.084960,0.032720,0.099061


### Conclusões - Internet

Nesta seção, observamos que a distribuição de dispositivos por tipo e faixa de velocidade evidenciam que a banda larga fixa é, em média, mais rápida que as redes móveis; e que a maioria dos dispositivos observados nas faixas mais baixas de velocidade é do tipo rede móvel. Isto corrobora as estimativas publicadas pelas plataformas [Statista](https://www.statista.com/statistics/1135124/brazil-internet-speed/) e [Speedtest](https://www.speedtest.net/global-index/brazil), que afirmam que as redes fixas de fato apresentam desempenho médio superior às redes móveis. Geograficamente, os acessos de velocidade superior estão concentrados nas regiões Sul e Sudeste; esta última região, inclusive, possui aproximadamente o triplo de dispositivos por habitante nas faixas superiores de velocidade (15 a 25, 25+ Mbps) se comparada às regiões Norte e Nordeste.

Além disso, estimamos a contagem de operadoras únicas de Internet por região como *proxy* para a infraestrutura de comunicações disponível em cada município. Ao agregarmos em nível regional, novamente nota-se que, caracteristicamente, municípios da região Sudeste possuem mais opções de prestadores de serviço do que municípios de outras regiões – em especial da Norte; entre as demais, as diferenças são menos significativas. Assim, os dados apontam para a possibilidade de que a região Norte esteja aquém das demais em termos de infraestrutura. De fato, considerados os desafios geográficos fisicamente impostos pela região, este seria o resultado esperado.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.1](#1.-Dados-de-Internet)**

## Dados de educação

### Visão geral

As colunas do conjunto de variáveis de educação são todas numéricas e contínuas. Aqui, cada sufixo indica o nível de educação considerado:

| Sufixo  | Nível de educação     | Anos escolares     |
| :------ | --------------------- | ------------------ |
| `_ef_1` | Ensino Fundamental I  | 1º ao 5º ano       |
| `_ef_2` | Ensino Fundamental II | 6º ao 9º ano       |
| `_em`   | Ensino Médio          | 1ª, 2ª e 3ª séries |

**Taxa de aprovação (Ens. Fund. I, Ens. Fund. II, Ens. Médio):** observações em pontos percentuais, com uma casa decimal, no intervalo [0.0, 100.0].

**SAEB:** observações representam a nota média, na [escala SAEB](https://academia.qedu.org.br/prova-brasil/454-2/), que varia de acordo com a disciplina e nível de ensino. A tabela abaixo resume as faixas de valores possíveis.

| Nível de ensino       | Língua Portuguesa | Matemática |
| :-------------------- | ----------------- | ---------- |
| Ensino Fundamental I  | 0 a 350           | 125 a 350  |
| Ensino Fundamental II | 200 a 400         | 200 a 425  |
| Ensino Médio          | 225 a 425         | 225 a 475  |

**Ideb:** observações referem-se ao Índice de Desenvolvimento da Educação Básica e admitem valores entre 0 e 10, com uma casa decimal, para todos os níveis de educação.

In [11]:
exploratory_analysis.get_dataset_description(dataset, 'education')

,tx_aprov_ef_1,saeb_mat_ef_1,saeb_port_ef_1,ideb_ef_1,tx_aprov_ef_2,saeb_mat_ef_2,saeb_port_ef_2,ideb_ef_2,tx_aprov_em,saeb_mat_em,saeb_port_em,ideb_em
count,3503.000000,3471.000000,3471.000000,3471.000000,3513.000000,3421.000000,3421.000000,3421.000000,3510.00000,3149.000000,3149.000000,3149.000000
mean,94.396717,225.827035,211.697560,5.817776,91.869912,258.889977,255.133213,4.661444,86.08339,274.558263,274.494287,4.070117
std,6.070335,22.068236,19.906565,0.912929,6.113734,17.742040,15.272435,0.714934,8.70051,19.473577,16.398680,0.640029
min,55.900000,159.770000,147.290000,3.300000,52.200000,206.150000,204.260000,2.300000,53.40000,210.860000,221.100000,2.000000
25%,91.300000,209.910000,197.640000,5.100000,88.400000,247.030000,245.090000,4.200000,80.50000,261.700000,263.690000,3.600000
50%,96.500000,226.740000,214.560000,5.900000,93.000000,259.490000,256.240000,4.700000,87.30000,275.940000,276.170000,4.200000
75%,99.200000,241.820000,226.445000,6.500000,96.400000,270.360000,265.520000,5.200000,93.20000,287.660000,286.030000,4.600000
max,100.000000,319.480000,291.330000,9.400000,100.000000,349.210000,315.360000,7.700000,100.00000,342.270000,322.840000,5.800000


### Conclusões - Educação

À exceção do indicador de Taxas de Aprovação, as variáveis observadas tem formato aproximadamente de sino aparentemente bem distribuída ao redor da média – linha pontilhada em vermelho. 

Como um todo, observamos que:

1. Os indicadores *deterioram-se* à medida em que se avança pelos níveis de educação

A concentração de pontos desloca-se à esquerda do eixo x entre o Ensino Fundamental I e Ensino Médio.

2. Ao observamos as caudas da distribuição por região – isto é, a contagem de escolas por região cujo desempenho seja superior a 90% ou inferior a 10% das demais do país –, notamos que as escolas de maior desempenho estão fortemente concentradas no eixo sul-sudeste, e as de menor desempenho, no eixo norte-nordeste.    

Em alguns indicadores, a região norte sequer possui cidades de média suficientemente grande para que se enquadre no decil superior. Isto acontece mais frequentemente ao observarmos os dados referentes ao ensino básico.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.2](#2.-Dados-de-Educa%C3%A7%C3%A3o)**

## Indicadores sociais

### Visão geral

As colunas do conjunto de indicadores sociais são todas numéricas e contínuas. Suas unidades de medida são

| Variável  | Unidade de medida     | Observações     |
| :-------- | --------------------- | ------------------ |
| `populacao` | Número de indivíduos | Dados do Censo de 2010 |
| `exp_vida`  | Média de anos | Esperança de vida ao nascer |
| `analfabetismo` | Analfabetos por 1000 habitantes | Pessoas que de 15 anos ou mais que não sabem ler ou escrever ao menos um bilhete simples |
| `tx_pobreza` | Percentual da população | Pessoas vivendo com menos de USD 5,5 PPC (paridade de poder de compra) por mês |
| `pib_per_capita` | Milhares de BRL por habitante | Em BRL (2016) |
| `mortalidade_inf` | Mortalidade de crianças por 1000 habitantes | Considera crianças que morreram antes de completar 1 ano de vida | 
| `maternidade_inf` | Percentual de meninas que tiveram filhos | Meninas de 10 a 14 anos de idade |

### Conclusões - Indicadores sociais

Os indicadores sociais apresentados servirão de variáveis de controle para a regressão. Antes que apresentemos qualificações sobre as regiões, é necessário frisar que os resultados apresentam certo viés que favorece as regiões Sul e Sudeste: como consideramos apenas as cidades que possuam ao menos 1 registro de teste de velocidade – e dada a concentração da infraestrutura de Internet nessa região –, existirão mais observações para cidades pertencentes a este eixo. Assim, os conclusões devem ser interpretadas com cautela.

Em termos de indicadores, nota-se que o Sudeste, além de concentrar a maior população considerada, também concentra os melhores resultados em todos os indicadores considerados, e o Sul tem desempenho similar. As regiões Norte e Nordeste, por outro lado, encontram-se na situação oposta: seus resultados são todos inferiores às regiões anteriores. A isto certamente está associada a distribuição de riqueza per capita no país, que está diretamente correlacionada ao bom desempenho nestes indicadores. Como caso atípico ao padrão constatado, temos apenas a expectativa de vida: apesar de todas as disparidades pontuadas, a esperança de vida média é muito similar entre todas as regiões – e a amplitude de observações é menor que a das demais variáveis.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.3](#3.-Indicadores-Sociais)**

### Correlações entre Internet e Educação

Partimos da hipótese de que a maior prevalência de dispositivos em faixas superiores de velocidade de acesso à Internet contribui positivamente para indicadores educacionais, em especial aos que se referem ao Ensino Médio – dada a maior autonomia exigida dos estudantes deste nível. Assim, analisamos a matriz de correlações e constatamos que a relação entre variáveis de acesso à Internet e indicadores educacionais é muito pouco significativa – não há correlação absoluta que supere o patamar de **0.1** entre variáveis destes grupos. De fato, as correlações mais significativas são observadas apenas em variáveis intragrupais, conforme esperado.

A seguir, respectivamente, a matriz de correlações e sua representação gráfica.

In [ ]:
exploratory_analysis.get_internet_education_corr(dataset)

# Testes de hipótese

## Diferenças entre taxas de aprovação por região

Conduzimos uma série de Testes A/B a fim de determinar se há diferença estatística significativa entre taxas de aprovação – consideradas todos os níveis de educação – de escolas localizadas na região Sudeste se comparadas às escolas das demais regiões. Com base nos intervalos de confiança gerados, pode-se afirmar que

* Existe evidência estatística de que as taxas de aprovação do Ensino Fundamental I e Ensino Médio do Sudeste são maiores do que as mesmas taxas de todas as demais regiões
* Para o Ensino Fundamental II, há evidência significativa de que as taxas de aprovação de escolas do Sudeste superem todas as demais regiões, **exceto o Centro-Oeste**

**Os intervalos de confiança para este teste estão disponíveis no [Apêndice C.1](#1.-Estat%C3%ADstica-do-Teste-A/B:-diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-por-estado)**

## Diferenças entre taxas de aprovação de municípios pobres e ricos

## Diferenças entre taxas de aprovação entre municípios com menor e maior acesso à Internet de alta velocidade

# Regressões

## Regressões univariadas

## Regressões multivariadas

# Conclusões

# Apêndices

## A. Limpeza e manipulação das bases brutas

## B. Gráficos de análise exploratória

### 1. Dados de Internet

Clique [aqui](#Conclus%C3%B5es---Internet) para retornar à leitura.

#### Distribuição de dispositivos/habitante por faixa de velocidade, por região

In [ ]:
exploratory_analysis.get_internet_plot(dataset, 'device_distribution')

#### Distribuição da média de operadoras, por região

In [ ]:
exploratory_analysis.get_internet_plot(dataset, 'isp_distribution')

### 2. Dados de Educação

Clique [aqui](#Conclus%C3%B5es---Educa%C3%A7%C3%A3o) para retornar à leitura.

#### IDEB

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'ideb')

#### Taxas de aprovação

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'approval_rate')

#### SAEB - Matemática

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'saeb_mat')

#### SAEB - Língua Portuguesa

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'saeb_port')

### 3. Indicadores Sociais

Clique [aqui](#Conclus%C3%B5es---Indicadores-sociais) para retornar à leitura.

#### População por região

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'population')

#### Expectativa de vida

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'life_expectancy')

#### Analfabetismo

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'analfabetism')

#### Taxa de pobreza

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'poverty')

#### PIB per capita

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'gdp_per_capita')

#### Mortalidade infantil

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'child_mortality')

#### Gravidez na adolescência

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'adolescent_pregnancy')

## C. Testes de Hipótese

### 1. Estatística do Teste A/B: diferenças entre taxas de aprovação por estado

Clique [aqui](#Diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-por-regi%C3%A3o) para voltar à leitura.

#### Ensino Fundamental I

In [45]:
linear_regression.print_approval_rate_comparison(dataset, 'tx_aprov_ef_1')

Existe evidência de que a taxa de aprovação nos anos iniciais do ensino fundamental no Sudeste é maior do que no Centro-Oeste.
Intervalos de confiança:
 Centro-Oeste		 [96.34818904397757, 96.98435812583394] 
 Sudeste		 [97.25856433579618, 97.59221466016437] 

Existe evidência de que a taxa de aprovação nos anos iniciais do ensino fundamental no Sudeste é maior do que no Nordeste.
Intervalos de confiança:
 Nordeste		 [90.10349360177604, 90.81179535213771] 
 Sudeste		 [97.25856433579618, 97.59221466016437] 

Existe evidência de que a taxa de aprovação nos anos iniciais do ensino fundamental no Sudeste é maior do que no Norte.
Intervalos de confiança:
 Norte		 [88.91512015834472, 90.62064626501292] 
 Sudeste		 [97.25856433579618, 97.59221466016437] 

Existe evidência de que a taxa de aprovação nos anos iniciais do ensino fundamental no Sudeste é maior do que no Sul.
Intervalos de confiança:
 Sul		 [95.21868220230202, 95.72711053463361] 
 Sudeste		 [97.25856433579618, 97.59221466016437] 



#### Ensino Fundamental II

In [46]:
linear_regression.print_approval_rate_comparison(dataset, 'tx_aprov_ef_2')

Não existe evidência de que a taxa de aprovação nos anos finais do ensino fundamental no Sudeste é maior do que no Centro-Oeste.
Intervalos de confiança:
 Centro-Oeste		 [92.37292559028768, 93.48716830642607] 
 Sudeste		 [92.37974924299355, 92.89997425930984] 

Existe evidência de que a taxa de aprovação nos anos finais do ensino fundamental no Sudeste é maior do que no Nordeste.
Intervalos de confiança:
 Nordeste		 [90.73241279371781, 91.45294591784919] 
 Sudeste		 [92.37974924299355, 92.89997425930984] 

Existe evidência de que a taxa de aprovação nos anos finais do ensino fundamental no Sudeste é maior do que no Norte.
Intervalos de confiança:
 Norte		 [89.1339487617496, 90.49059997471238] 
 Sudeste		 [92.37974924299355, 92.89997425930984] 

Existe evidência de que a taxa de aprovação nos anos finais do ensino fundamental no Sudeste é maior do que no Sul.
Intervalos de confiança:
 Sul		 [91.58414348282209, 92.1725466580232] 
 Sudeste		 [92.37974924299355, 92.89997425930984] 



#### Ensino Médio

In [47]:
linear_regression.print_approval_rate_comparison(dataset, 'tx_aprov_em')

Existe evidência de que a taxa de aprovação no ensino médio no Sudeste é maior do que no Centro-Oeste.
Intervalos de confiança:
 Centro-Oeste		 [83.47058613060253, 85.58309557961132] 
 Sudeste		 [88.38377258303939, 89.09042096534785] 

Existe evidência de que a taxa de aprovação no ensino médio no Sudeste é maior do que no Nordeste.
Intervalos de confiança:
 Nordeste		 [84.37589254261596, 85.34355190182839] 
 Sudeste		 [88.38377258303939, 89.09042096534785] 

Existe evidência de que a taxa de aprovação no ensino médio no Sudeste é maior do que no Norte.
Intervalos de confiança:
 Norte		 [81.25064751829753, 83.04682540589017] 
 Sudeste		 [88.38377258303939, 89.09042096534785] 

Existe evidência de que a taxa de aprovação no ensino médio no Sudeste é maior do que no Sul.
Intervalos de confiança:
 Sul		 [83.8299580596733, 84.72316969363051] 
 Sudeste		 [88.38377258303939, 89.09042096534785] 

